# Analyse et nettoyage des données Lycées du site datagouv

In [1]:
# import bibliothéques d'analyse de python
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import FuncFormatter
import seaborn as sns
import numpy as np

import fonctions.perso_stats as perso_stats


In [2]:
# import des fichiers datagouv

# effectif_lycee_sexe = pd.read_csv('donnee/fr-en-lycee_gt-effectifs-niveau-sexe-lv.csv',sep=';') # bof
effectif_lycee_spe2 = pd.read_csv('donnee/fr-en-effectifs-specialites-doublettes-terminale-generale.csv',sep=';') # fichier interessant
effectif_lycee_pro = pd.read_csv('donnee/fr-en-lycee_pro-effectifs-niveau-sexe-mef.csv',sep=';') # fichier interessant
ips_lycee = pd.read_csv('donnee/fr-en-ips-lycees-ap2022.csv',sep=';') # fichier interessant

In [3]:
ips_lycee.head()

,rentree_scolaire,academie,code_du_departement,departement,uai,nom_de_l_etablissment,code_insee_de_la_commune,nom_de_la_commune,secteur,type_de_lycee,effectifs_voie_gt,effectifs_voie_pro,effectifs_ensemble_gt_pro,ips_voie_gt,ips_voie_pro,ips_ensemble_gt_pro,ecart_type_de_l_ips_voie_gt,ecart_type_de_l_ips_voie_pro
0,2022-2023,LYON,001,AIN,0010001W,LYCEE PROFESSIONNEL ALEXANDRE BERARD LYCEE DES...,01004,AMBERIEU EN BUGEY,public,LP,NaN,583.0,583.0,NaN,91.5,91.5,NaN,27.2
1,2022-2023,LYON,001,AIN,0010006B,LYCEE POLYVALENT SAINT EXUPERY LYCEE DES METIE...,01033,VALSERHONE,public,LPO,556.0,334.0,890.0,99.2,84.2,93.6,33.3,24.8
2,2022-2023,LYON,001,AIN,0010013J,LYCEE GENERAL LALANDE,01053,BOURG EN BRESSE,public,LEGT,1028.0,NaN,1028.0,123.0,NaN,123.0,34.7,NaN
3,2022-2023,LYON,001,AIN,0010016M,LYCEE POLYVALENT JOSEPH MARIE CARRIAT,01053,BOURG EN BRESSE,public,LPO,1362.0,335.0,1697.0,111.7,98.8,109.2,32.9,30.2
4,2022-2023,LYON,001,AIN,0010069V,LYCEE GENERAL ET TECHNOLOGIQUE PRIVE ENSEMBLE ...,01034,BELLEY,privé sous contrat,LEGT,250.0,NaN,250.0,115.8,NaN,115.8,27.5,NaN


In [4]:
ips_lycee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3598 entries, 0 to 3597
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   rentree_scolaire              3598 non-null   object 
 1   academie                      3598 non-null   object 
 2   code_du_departement           3598 non-null   object 
 3   departement                   3598 non-null   object 
 4   uai                           3598 non-null   object 
 5   nom_de_l_etablissment         3598 non-null   object 
 6   code_insee_de_la_commune      3598 non-null   object 
 7   nom_de_la_commune             3598 non-null   object 
 8   secteur                       3598 non-null   object 
 9   type_de_lycee                 3598 non-null   object 
 10  effectifs_voie_gt             2464 non-null   float64
 11  effectifs_voie_pro            2075 non-null   float64
 12  effectifs_ensemble_gt_pro     3598 non-null   float64
 13  ips

In [5]:
effectif_lycee_pro.head()

,rentree_scolaire,academie_2020,academie_2020_lib_l,numero_d_etablissement,patronyme,adresse_condensee,code_postal,commune_d_implantation,commune_d_implantation_lib_l,commune_et_arrondissement,commune_et_arrondissement_lib_l,secteur_d_enseignement_lib_l,mef_bcp_11,mef_bcp_11_lib_l,mef_bcp_6,mef_bcp_6_lib_l,nombre_d_eleves_total,nombre_d_eleves_filles,nombre_d_eleves_garcons
0,2019,18,ORLEANS-TOURS,0410959V,CAMILLE CLAUDEL,C CLAUDEL 10 RUE ALBERT CAMUS,41018,41018,BLOIS,41018,BLOIS,SECTEUR PUBLIC,32211033107,1BTS2 OPTICIEN LUNETIER,322110,BTS EN 2 ANS : 1ERE ANNEE,32,28.0,4.0
1,2019,18,ORLEANS-TOURS,0450029M,BERNARD PALISSY,B PALISSY RUE DU 32E R I,45502,45155,GIEN,45155,GIEN,SECTEUR PUBLIC,32221031209,2BTS2 MANAGEMENT UNITES COMMERCIALES,322210,BTS EN 2 ANS : 2EME ANNEE,16,13.0,3.0
2,2019,18,ORLEANS-TOURS,0450040Z,EN FORET,EN FORET AVENUE CHAUTEMPS,45207,45208,MONTARGIS,45208,MONTARGIS,SECTEUR PUBLIC,32221031408,2BTS2 COMPTABILITE ET GESTION,322210,BTS EN 2 ANS : 2EME ANNEE,19,14.0,5.0
3,2019,18,ORLEANS-TOURS,0450042B,DURZY,DURZY 23 RUE L DE VINCI,45702,45338,VILLEMANDEUR,45338,VILLEMANDEUR,SECTEUR PUBLIC,32221020112,2BTS2 SYST.NUMER. OPT.A INFORM.&RESEAUX,322210,BTS EN 2 ANS : 2EME ANNEE,15,2.0,13.0
4,2019,18,ORLEANS-TOURS,0450043C,JEANNETTE VERDIER,J VERDIER RUE CYRILLE ROBERT,45207,45208,MONTARGIS,45208,MONTARGIS,SECTEUR PUBLIC,23210024240,1CAP2 METIERS DE LA MODE-V-TEMENT FLOU,232100,CAP EN 2 ANS : 1ERE ANNEE,24,22.0,2.0


In [6]:
effectif_lycee_pro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221763 entries, 0 to 221762
Data columns (total 19 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   rentree_scolaire                 221763 non-null  int64  
 1   academie_2020                    221763 non-null  int64  
 2   academie_2020_lib_l              221763 non-null  object 
 3   numero_d_etablissement           221763 non-null  object 
 4   patronyme                        221637 non-null  object 
 5   adresse_condensee                218896 non-null  object 
 6   code_postal                      221763 non-null  int64  
 7   commune_d_implantation           221763 non-null  object 
 8   commune_d_implantation_lib_l     221763 non-null  object 
 9   commune_et_arrondissement        221763 non-null  object 
 10  commune_et_arrondissement_lib_l  221763 non-null  object 
 11  secteur_d_enseignement_lib_l     221763 non-null  object 
 12  me

In [7]:
effectif_lycee_spe2.head()

,rentree_scolaire,region_academique,academie,departement,commune,numero_etablissement,denomination,patronyme,secteur,effectif_total_en_terminale_generale,...,0012_humanites_litterature_et_philosophie_sciences_economiques_et_sociales_garcons,0013_hist_geo_geopolitique_sc_politiques_mathematiques_filles,0013_hist_geo_geopolitique_sc_politiques_mathematiques_garcons,0014_mathematiques_sciences_inge_et_phy_filles,0014_mathematiques_sciences_inge_et_phy_garcons,0015_hist_geo_geopolitique_sc_politiques_sciences_de_la_vie_et_de_la_terre_filles,0015_hist_geo_geopolitique_sc_politiques_sciences_de_la_vie_et_de_la_terre_garcons,autres_combinaisons_filles,autres_combinaisons_garcons,code_postal
0,2020,AUVERGNE-RHONE-ALPES,CLERMONT-FERRAND,CANTAL,SAINT-FLOUR,0150030B,LYCEE POLYVALENT,DE HAUTE AUVERGNE,PUBLIC,113,...,0,2,0,0,0,0,3,4,2,15101.0
1,2020,AUVERGNE-RHONE-ALPES,CLERMONT-FERRAND,HAUTE-LOIRE,YSSINGEAUX,0430065M,LYCEE GEN.ET TECHNOL.PRIVE,SAINT-GABRIEL,PRIVE SOUS CONTRAT,64,...,0,1,0,0,0,3,1,6,4,43200.0
2,2020,AUVERGNE-RHONE-ALPES,GRENOBLE,ARDECHE,AUBENAS,0070004S,LYCEE POLYVALENT,ASTIER,PUBLIC,37,...,0,0,0,3,14,0,0,0,0,7205.0
3,2020,AUVERGNE-RHONE-ALPES,GRENOBLE,ARDECHE,LE CHEYLARD,0071397F,LYCEE POLYVALENT,LE CHEYLARD,PUBLIC,98,...,0,1,0,0,0,1,1,7,4,7160.0
4,2020,AUVERGNE-RHONE-ALPES,GRENOBLE,DROME,ROMANS-SUR-ISERE,0260022H,LYCEE GENERAL,ALBERT TRIBOULET,PUBLIC,324,...,0,3,1,0,0,2,3,10,15,26102.0


In [8]:
effectif_lycee_spe2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6740 entries, 0 to 6739
Data columns (total 83 columns):
 #   Column                                                                                 Non-Null Count  Dtype  
---  ------                                                                                 --------------  -----  
 0   rentree_scolaire                                                                       6740 non-null   int64  
 1   region_academique                                                                      6740 non-null   object 
 2   academie                                                                               6740 non-null   object 
 3   departement                                                                            6740 non-null   object 
 4   commune                                                                                6740 non-null   object 
 5   numero_etablissement                                                        

In [14]:
#renommer uai par numero d'établissiment du fichier ips pour rester coherent entre les fichiers
ips_lycee.rename(columns={'uai':'numero_etablissement'},inplace=True)

In [19]:
# voir pour modifier la rentré scolaire du fichier ips par soucis de coherence entre les fichiers
ips_lycee['rentree_scolaire'] = ips_lycee.rentree_scolaire.map({'2022-2023':2022})

In [23]:
# voir pour changer academie 2020 du lycee pro par academie
effectif_lycee_pro.rename(columns={'academie_2020_lib_l' : "academie"},inplace=True)

In [12]:
# harmonisé les colonnes secteurs public privée de tout les fichiers